In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import math

In [ ]:
df = pd.read_excel('/content/Data Model - Pizza Sales.xlsx')

In [ ]:
df.to_parquet('pizza_dataset.parquet')

In [ ]:
df = pd.read_parquet('pizza_dataset.parquet')

In [ ]:
df.head()

In [ ]:
df.drop(columns=['order_details_id', 'order_id', 'pizza_id', 'order_time', 'pizza_category'], inplace= True)

In [ ]:
df.order_date = pd.to_datetime(df.order_date)

In [ ]:
df.head()

In [ ]:
df = df.groupby(['pizza_name', 'order_date' , 'pizza_size' , 'pizza_ingredients', 'unit_price'])['quantity'].sum().reset_index()

# Extract Ingredients

In [ ]:
def build_detailed_recipes(df, sizes=('S','M','L')):
    size_mult = {'S': 0.7, 'M': 1.0, 'L': 1.4}
    dough_by_size = {'S': 180, 'M': 260, 'L': 350}
    cheese_overrides = {
        'Mozzarella Cheese': 120, 'Provolone Cheese': 60, 'Smoked Gouda Cheese': 60, 'Romano Cheese': 40,
        'Blue Cheese': 40, 'Brie Carre Cheese': 50, 'Fontina Cheese': 60, 'Gouda Cheese': 60,
        'Asiago Cheese': 50, 'Parmigiano Reggiano Cheese': 25, 'Ricotta Cheese': 60, 'Feta Cheese': 60,
        'Goat Cheese': 50, 'Gorgonzola Piccante Cheese': 50
    }
    sauce_overrides = {
        'Barbecue Sauce': 50, 'Alfredo Sauce': 60, 'Pesto Sauce': 40, 'Chipotle Sauce': 40, 'Thai Sweet Chilli Sauce': 40
    }
    meat_overrides = {
        'Pepperoni': 70, 'Bacon': 50, 'Chicken': 90, 'Prosciutto': 40, 'Sliced Ham': 50, 'Chorizo Sausage': 80,
        'Italian Sausage': 80, 'Pancetta': 40, 'Calabrese Salami': 80, 'Capocollo': 70, 'Genoa Salami': 70,
        'Soppressata Salami': 80, 'Coarse Sicilian Salami': 80, 'Luganega Sausage': 80, 'Barbecued Chicken': 90,
        'Beef Chuck Roast': 90, 'Prosciutto di San Daniele': 40, '慛duja Salami': 70
    }
    veg_overrides = {
        'Tomatoes': 60, 'Plum Tomatoes': 60, 'Red Peppers': 40, 'Green Peppers': 40, 'Red Onions': 30, 'Onions': 30,
        'Mushrooms': 70, 'Spinach': 40, 'Corn': 40, 'Artichoke': 40, 'Artichokes': 40, 'Eggplant': 60, 'Zucchini': 40,
        'Pineapple': 80, 'Jalapeno Peppers': 25, 'Green Olives': 30, 'Kalamata Olives': 30, 'Sun-dried Tomatoes': 25,
        'Cilantro': 5, 'Arugula': 20, 'Pears': 50, 'Peperoncini verdi': 20, 'Anchovies': 15, 'Garlic': 5,
        'Caramelized Onions': 30
    }

    def get_base_qty(ing):
        il = ing.strip()
        if il in cheese_overrides:
            return cheese_overrides[il]
        if il in sauce_overrides:
            return sauce_overrides[il]
        if il in meat_overrides:
            return meat_overrides[il]
        if il in veg_overrides:
            return veg_overrides[il]
        l = il.lower()
        if 'cheese' in l:
            return 80
        if 'sauce' in l or any(k in l for k in ['chipotle', 'pesto', 'alfredo', 'thai', 'barbecue']):
            return 50
        for k in ['chicken', 'bacon', 'salami', 'sausage', 'pepperoni', 'ham', 'prosciutto', 'beef', 'pancetta', 'capocollo']:
            if k in l:
                return 70
        for k in ['pepper', 'tomato', 'mushroom', 'onion', 'spinach', 'corn', 'artich', 'eggplant', 'pineapple', 'olive', 'cilantro', 'zucchini', 'pear', 'anchovi']:
            if k in l:
                return 40
        if 'garlic' in l:
            return 5
        return 30

    pizza_ingredients = (
        df.groupby("pizza_name")["pizza_ingredients"]
        .apply(lambda x: set(", ".join(x).split(", ")))
        .to_dict()
    )

    recipes = {}
    for pizza, ings in pizza_ingredients.items():
        recipes[pizza] = {}
        for size in sizes:
            recipes[pizza][size] = {}
            recipes[pizza][size]['dough'] = dough_by_size[size]
            for ing in sorted(ings):
                base = get_base_qty(ing)
                qty = int(round(base * size_mult[size]))
                recipes[pizza][size][ing] = qty
    return recipes


In [ ]:
recipes = build_detailed_recipes(df)

In [ ]:
recipes

In [ ]:
df.drop(columns = ['pizza_ingredients'], inplace = True)

In [ ]:
df = df[df['pizza_size'].isin(['S', 'M', 'L'])]

# Handling Time-Series Data

In [ ]:
def calculate_demand_per_day(df, recipes, start_date='2015-01-01', period=1):
    start_date = pd.to_datetime(start_date)
    end_date = start_date + timedelta(days=period)
    df_period = df[(df['order_date'] >= start_date) & (df['order_date'] < end_date)]

    pizza_demand = df_period.groupby(['order_date', 'pizza_name', 'pizza_size']).agg({'quantity': 'sum'}).reset_index()

    demand_per_day = {}
    for _, row in pizza_demand.iterrows():
        pizza, size, quantity = row['pizza_name'], row['pizza_size'], row['quantity']
        recipe = recipes[pizza][size]  # recipe is a dict {ingredient: qty_per_pizza}

        for ingredient, qty_per_pizza in recipe.items():
            if ingredient not in demand_per_day:
                demand_per_day[ingredient] = 0
            demand_per_day[ingredient] += quantity * qty_per_pizza

    return demand_per_day


In [ ]:
def compute_stock_from_demand(df, recipes, ingredient='all', start_date='2015-01-01', window_days=14, multiplier=0.5):

    demand = calculate_demand_per_day(df, recipes, start_date=start_date, period=window_days)

    if ingredient == 'all':
        for ing in demand:
            demand[ing] *= multiplier
        return demand
    else:
        if ingredient in demand:
            return {ingredient: demand[ingredient] * multiplier}
        else:
            return {ingredient: 0}  # return 0 if ingredient not in demand


In [ ]:
initial_stock = compute_stock_from_demand(df, recipes,ingredient='all', window_days=14, multiplier= 0.5)

In [ ]:
def optimize_multipliers(multipliers, avg_demand, waste, stockout):
    for ing in multipliers:
        if stockout[ing] > 0:
            multipliers[ing] *= 1 + min(0.5, stockout[ing] / (avg_demand[ing] * 7 + 1e-6))
        elif waste[ing] > 0:
            multipliers[ing] *= 1 - min(0.5, waste[ing] / (avg_demand[ing] * 7 + 1e-6))
    return multipliers

# Simulation

In [ ]:
def run_simulation(df, recipes, initial_stock, start_date='2015-01-01', history_days=28):
    start_date = pd.to_datetime(start_date)

    # Initialize stock and multipliers
    stock = initial_stock.copy()
    multipliers = {ing: 1.0 for ing in stock.keys()}

    week = 0
    history = []

    while True:
        week += 1
        print(f"\n=== Week {week} Report ===")

        inventory_start = stock.copy()

        demand_week = calculate_demand_per_day(df, recipes, start_date=start_date, period=7)

        waste = {}
        stockout = {}
        demand_fulfilled = {}
        multipliers_prev = multipliers.copy()

        for ing in multipliers:
            available = stock.get(ing, 0)
            required = demand_week.get(ing, 0)
            if available >= required:
                demand_fulfilled[ing] = required
                stock[ing] -= required
                stockout[ing] = 0
            else:
                demand_fulfilled[ing] = available
                stockout[ing] = required - available
                stock[ing] = 0

        for ing in multipliers:
            waste[ing] = stock.get(ing, 0)

        history.append(demand_week)
        if len(history) > history_days // 7:  # history in weeks
            history.pop(0)

        avg_demand = {}
        for ing in multipliers:
            total = sum(h.get(ing, 0) for h in history)
            avg_demand[ing] = total / 7 / len(history) if history else 0

        multipliers = optimize_multipliers(multipliers, avg_demand, waste, stockout)

        shopping_list = {ing: avg_demand[ing] * 7 * multipliers[ing] for ing in multipliers}
        for ing, qty in shopping_list.items():
            stock[ing] = qty  # stock for next week = shopping list

        print(f"Inventory at start: {inventory_start}")
        print(f"Demand this week: {demand_week}")
        print(f"Demand fulfilled: {demand_fulfilled}")
        print(f"Stockout (lost demand): {stockout}")
        print(f"Waste (expired stock): {waste}")

        print("\n--- Multipliers ---")
        for ing in multipliers:
            print(f"{ing}: prev={multipliers_prev[ing]:.2f}, new={multipliers[ing]:.2f}")

        print(f"\nShopping list for next week: {shopping_list}")

        start_date += timedelta(days=7)

        cont = input("Press Enter to simulate next week (or type 'q' to quit): ")
        if cont.lower() == 'q':
            break


In [ ]:
run_simulation(df, recipes, initial_stock, start_date='2015-01-01', history_days=366)